<a href="https://colab.research.google.com/github/GirishKGit/CrewAI/blob/main/AI_Conversational_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install datasets


In [4]:
!pip install convokit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.2/183.2 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.1 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-3.0.0-py3-none-any.whl size=216707 sha256=01c4e42b143fb45e685119e62bea44fdf6c53d1a5cc778e78a93ac757b27a9ed
  Stored in directory: /root/.cache/pip/wheels/c4/89/8c/2677fdb888588b6f93cb6ac86bdfb020f1f1c33e0d5525b231
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha

In [6]:
import os
import pandas as pd
from convokit import Corpus, download

def load_reddit_corpus(data_dir='/content', corpus_name='reddit-corpus-small'):
    """
    Downloads and loads the Reddit corpus. Re-usable function to download and load corpus.
    Args:
        data_dir (str): Directory where data will be saved.
        corpus_name (str): Name of the corpus to download.
    Returns:
        Corpus: The loaded Convokit corpus.
    """
    corpus_path = os.path.join(data_dir, corpus_name)

    # Check if corpus exists, download if not
    if not os.path.exists(corpus_path):
        print(f"Downloading {corpus_name} to {data_dir}...")
        corpus_path = download(corpus_name, data_dir=data_dir)
    else:
        print(f"Corpus already exists in {corpus_path}.")

    # Load the corpus
    corpus = Corpus(filename=corpus_path)
    return corpus

def extract_utterances(corpus):
    """
    Extracts relevant utterance data from a given corpus and returns it as a pandas DataFrame.
    Args:
        corpus (Corpus): A Convokit corpus.
    Returns:
        pd.DataFrame: DataFrame with utterance_id, speaker_id, conversation_id, text, timestamp, reply_to.
    """
    utterances_data = []

    # Extract utterances and relevant information
    for utterance in corpus.iter_utterances():
        utterances_data.append({
            'utterance_id': utterance.id,
            'speaker_id': utterance.speaker.id,
            'conversation_id': utterance.conversation_id,
            'text': utterance.text,
            'timestamp': utterance.timestamp,
            'reply_to': utterance.reply_to if utterance.reply_to else 'No reply'
        })

    # Convert to a DataFrame
    utterances_df = pd.DataFrame(utterances_data)
    return utterances_df

# Example usage:
corpus = load_reddit_corpus()  # Load the Reddit corpus
utterances_df = extract_utterances(corpus)  # Extract utterance data
print(utterances_df.head())  # Display the first few rows of the DataFrame


  utterance_id     speaker_id conversation_id  \
0       9c716m  AutoModerator          9c716m   
1       9c8amk         rt2236          9c8amk   
2       9ca0yk    Im_Tsuikyit          9ca0yk   
3       9cfz10  AutoModerator          9cfz10   
4       9crqp2  mamypokopants          9crqp2   

                                                text   timestamp  reply_to  
0  Talk about your day. Anything goes, but subred...  1535839576  No reply  
1  I went to visit a few days ago and Ioved it. I...  1535850650  No reply  
2  One time, my family and I had just returned fr...  1535868569  No reply  
3  Talk about your day. Anything goes, but subred...  1535925965  No reply  
4                                                     1536029129  No reply  


In [7]:
# Find rows with empty or missing text
empty_texts = utterances_df[utterances_df['text'].isnull() | (utterances_df['text'] == '')]
print(empty_texts)


       utterance_id     speaker_id conversation_id text   timestamp  reply_to
4            9crqp2  mamypokopants          9crqp2       1536029129  No reply
10           9drpte    RzrRainMnky          9drpte       1536302928  No reply
11           9dsufk   Garlicvideos          9dsufk       1536314439  No reply
13           9e35l8        dudez77          9e35l8       1536399829  No reply
18           9ehwa7   drewiepoodle          9ehwa7       1536538437  No reply
...             ...            ...             ...  ...         ...       ...
294120       9jwmgy       TheKleen          9jwmgy       1538227291  No reply
294121       9k0bpn          xFaro          9k0bpn       1538253478  No reply
294122       9k0usd     farway2000          9k0usd       1538257670  No reply
294123       9k1c5a       mattywoo          9k1c5a       1538261785  No reply
294124       9k4nb6    Dudehitscar          9k4nb6       1538294820  No reply

[4524 rows x 6 columns]


In [8]:
utterances_df = utterances_df[utterances_df['text'].notnull() & (utterances_df['text'] != '')]


In [9]:
utterances_df['text'].fillna("No text provided", inplace=True)
utterances_df['text'].replace('', "No text provided", inplace=True)


In [10]:
# Check if there are any rows with missing or empty text fields after cleaning
empty_texts_after_cleaning = utterances_df[utterances_df['text'].isnull() | (utterances_df['text'] == '')]
print(empty_texts_after_cleaning)


Empty DataFrame
Columns: [utterance_id, speaker_id, conversation_id, text, timestamp, reply_to]
Index: []


In [11]:
# Check for missing values in other important columns
print(utterances_df.isnull().sum())


utterance_id       0
speaker_id         0
conversation_id    0
text               0
timestamp          0
reply_to           0
dtype: int64


In [12]:
utterances_df['reply_to'].fillna("No reply", inplace=True)


In [13]:
utterances_df.head()

,utterance_id,speaker_id,conversation_id,text,timestamp,reply_to
0,9c716m,AutoModerator,9c716m,"Talk about your day. Anything goes, but subred...",1535839576,No reply
1,9c8amk,rt2236,9c8amk,I went to visit a few days ago and Ioved it. I...,1535850650,No reply
2,9ca0yk,Im_Tsuikyit,9ca0yk,"One time, my family and I had just returned fr...",1535868569,No reply
3,9cfz10,AutoModerator,9cfz10,"Talk about your day. Anything goes, but subred...",1535925965,No reply
5,9d07z8,AutoModerator,9d07z8,"Talk about your day. Anything goes, but subred...",1536098767,No reply


In [14]:
# Count the number of utterances in each conversation
conversation_summary = utterances_df.groupby('conversation_id').size().reset_index(name='utterance_count')
print(conversation_summary.head())


  conversation_id  utterance_count
0          9bypg3               20
1          9byqyq               11
2          9byrus               15
3          9bytc6               45
4          9byu1v               10


In [15]:
# Find conversation starters
conversation_starters = utterances_df[utterances_df['reply_to'] == "No reply"]
print(conversation_starters.head())


  utterance_id     speaker_id conversation_id  \
0       9c716m  AutoModerator          9c716m   
1       9c8amk         rt2236          9c8amk   
2       9ca0yk    Im_Tsuikyit          9ca0yk   
3       9cfz10  AutoModerator          9cfz10   
5       9d07z8  AutoModerator          9d07z8   

                                                text   timestamp  reply_to  
0  Talk about your day. Anything goes, but subred...  1535839576  No reply  
1  I went to visit a few days ago and Ioved it. I...  1535850650  No reply  
2  One time, my family and I had just returned fr...  1535868569  No reply  
3  Talk about your day. Anything goes, but subred...  1535925965  No reply  
5  Talk about your day. Anything goes, but subred...  1536098767  No reply  


In [16]:
# Count replies to each utterance
reply_count = utterances_df['reply_to'].value_counts().reset_index(name='reply_count')
reply_count.columns = ['utterance_id', 'reply_count']
print(reply_count.head())


  utterance_id  reply_count
0     No reply         3763
1      e5k5np8          472
2      e5ibybt          239
3      e5bp7cl          177
4      e5871fa          165


In [17]:
# Filter conversations where there is at least one "No reply" utterance
no_reply_conversations = utterances_df[utterances_df['reply_to'] == "No reply"]

# Group by conversation ID and count utterances per conversation
conversation_summary_no_reply = no_reply_conversations.groupby('conversation_id').size().reset_index(name='utterance_count')
print(conversation_summary_no_reply.head())

# View an example conversation with "No reply"
example_conversation_id = conversation_summary_no_reply.iloc[0]['conversation_id']
example_conversation = utterances_df[utterances_df['conversation_id'] == example_conversation_id]
print(example_conversation)


  conversation_id  utterance_count
0          9bypg3                1
1          9byrus                1
2          9bz3xn                1
3          9bz6r0                1
4          9bz763                1
      utterance_id            speaker_id conversation_id  \
55590       9bypg3          metalic_acid          9bypg3   
55665      e56qunv            spaceghoti          9bypg3   
55666      e56vt3t             Take_Beer          9bypg3   
55667      e56vwto            spaceghoti          9bypg3   
55668      e56x4f2             Take_Beer          9bypg3   
55669      e56x966            spaceghoti          9bypg3   
55670      e56xizr             Take_Beer          9bypg3   
55671      e56xorc            spaceghoti          9bypg3   
55672      e56xugz  midwesternphotograph          9bypg3   
55673      e56y86b             Take_Beer          9bypg3   
55674      e56yf18            spaceghoti          9bypg3   
55675      e56z3o2             Take_Beer          9bypg3   
55676     

In [18]:
!pip install detoxify


In [19]:
from detoxify import Detoxify

# Function to detect toxicity in user input
def detect_bias_or_toxicity(user_input):
    # Use Detoxify model to predict toxicity
    toxicity = Detoxify('original').predict(user_input)['toxicity']
    return toxicity > 0.7  # Threshold for toxicity detection

# Example chatbot function with bias detection
def chatbot(user_input):
    if detect_bias_or_toxicity(user_input):
        return "I can't comment on that."
    else:
        # Normal response generation (Cohere API or other)
        return generate_response(user_input)


In [20]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [21]:
import os
import cohere

# Set your Cohere API key as an environment variable
os.environ['COHERE_API_KEY'] = '74CedFbLCqLL4EK6R4onapiBrTUXJoknx59VrsKQ'

# Initialize the Cohere API with the key
co = cohere.Client(os.getenv("COHERE_API_KEY"))


In [22]:
!pip install sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 58.0 MB/s eta 0:00:00


In [23]:
# Embed Reddit Conversations Using Hugging Face Sentence Transformers
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Load the Hugging Face sentence transformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Sample data (Reddit conversation 'text' field from your dataset)
texts = [
    "Talk about your day. Anything goes, but subreddit rules still apply.",
    "I went to visit a few days ago and loved it.",
    "One time, my family and I had just returned from the beach."
]

# Generate embeddings
embeddings = model.encode(texts)

# Create a FAISS index to store embeddings
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)

# Add embeddings to the index
index.add(embeddings)

# Save embeddings and FAISS index for retrieval later
np.save('reddit_embeddings.npy', embeddings)

print("Embeddings generated and FAISS index created.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings generated and FAISS index created.


In [24]:
embeddings

array([[-0.04613591, -0.0034425 ,  0.03919309, ...,  0.02243084,
        -0.00262925, -0.00871499],
       [ 0.04835767, -0.00784416,  0.06136188, ...,  0.01608643,
        -0.05029471,  0.01860338],
       [ 0.01496075,  0.03046594,  0.09300791, ...,  0.00111784,
        -0.11325203,  0.02372815]], dtype=float32)

In [25]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np
import cohere

# Load Hugging Face model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Example conversation prompts (ensure these are properly defined)
prompts = [
    "You can't handle the truth!",
    "May the Force be with you.",
    "I'll be back.",
    "Here's looking at you, kid.",
    "Houston, we have a problem."
]

# Step 1: Generate embeddings for the prompts (this is necessary to create the FAISS index)
def embed_texts(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(cls_embedding)

    return embeddings

# Embed the prompts
embeddings = embed_texts(prompts)

# Step 2: Create FAISS index and add embeddings
embedding_dim = embeddings[0].shape[1]
index = faiss.IndexFlatL2(embedding_dim)

# Flatten embeddings for FAISS
embeddings = np.vstack(embeddings)
index.add(embeddings)

# Initialize Cohere client (ensure API key is set)
cohere_client = cohere.Client(os.getenv('COHERE_API_KEY'))

# Step 3: Function to embed user query
def embed_user_query(query, tokenizer, model, device):
    inputs = tokenizer(query, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        query_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()

    return query_embedding

# Step 4: Function to retrieve the closest conversation using FAISS
def retrieve_conversation(query, tokenizer, model, index, prompts, device):
    query_embedding = embed_user_query(query, tokenizer, model, device)

    # Perform FAISS search to get the closest match
    D, I = index.search(query_embedding, k=1)
    closest_conversation = prompts[I[0][0]]

    return closest_conversation

# Step 5: Function to generate a response using Cohere API
def generate_response(closest_conversation, user_query, cohere_client):
    prompt = f"Previous conversation: {closest_conversation}\nUser: {user_query}\nBot:"

    response = cohere_client.generate(
        model='command-r-plus-04-2024',
        prompt=prompt,
        max_tokens=100
    )

    return response.generations[0].text.strip()

# Example Usage:
user_input = "Tell me something interesting about space."
closest_conversation = retrieve_conversation(user_input, tokenizer, model, index, prompts, device)
bot_response = generate_response(closest_conversation, user_input, cohere_client)

print(f"User: {user_input}")
print(f"Bot: {bot_response}")


User: Tell me something interesting about space.
Bot: Did you know that the Milky Way and Andromeda galaxies are on a collision course? In about 4-5 billion years, the two galaxies will merge to form a giant elliptical galaxy. This event will likely have a profound impact on the structure of the universe and the fate of our solar system. It's a reminder of the dynamic and ever-changing nature of the cosmos!


In [26]:
import pickle
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch

# Load or define your model, tokenizer, and embedding
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Your text prompts and embeddings (make sure embeddings are already created)
prompts = ["example prompt 1", "example prompt 2", "example prompt 3"]
embeddings = np.random.rand(len(prompts), 384)  # Replace with your actual embeddings

# FAISS index creation (assuming embedding dimension is 384)
index = faiss.IndexFlatL2(384)
index.add(embeddings)

# Save the tokenizer, model, embeddings, FAISS index, and prompts to a pickle file in Colab
save_path = '/content/chatbot_state_v1.pkl'  # The file will be saved in Colab's local storage
with open(save_path, 'wb') as f:
    pickle.dump({
        'tokenizer': tokenizer,
        'model': model,
        'embeddings': embeddings,
        'faiss_index': index,
        'prompts': prompts
    }, f)

print(f"Chatbot state saved successfully at {save_path}.")

# Download the pickle file to your local system from Colab
from google.colab import files
files.download(save_path)


Chatbot state saved successfully at /content/chatbot_state_v1.pkl.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 321, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

In [27]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2


In [29]:
!pip install requests


In [30]:
import requests

# Your API key from NewsAPI
api_key = "6aaa6fe75c11492384c603cddab771ca"

# Define the endpoint and parameters for AI-related news
url = "https://newsapi.org/v2/everything"
parameters = {
    'q': 'artificial intelligence OR NLP OR computer vision',  # Query to search for AI, NLP, or Computer Vision news
    'sortBy': 'relevancy',  # Sort news by relevancy
    'apiKey': api_key  # Your API key
}

# Make the request
response = requests.get(url, params=parameters)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    articles = data['articles']

    # Display a summary of the articles
    for article in articles[:5]:  # Get the first 5 articles
        print(f"Title: {article['title']}")
        print(f"Source: {article['source']['name']}")
        print(f"Description: {article['description']}\n")
else:
    print("Error:", response.status_code)


Title: How AI Is Deciphering Lost Scrolls From the Roman Empire
Source: Gizmodo.com
Description: Scrolls found in the shadow of Vesuvius and libraries of ancient texts besides are being illuminated by machine learning and computer vision.

Title: Marvel’s Vision Series Is Bringing Back James Spader as Ultron
Source: Gizmodo.com
Description: The as-yet-untitled Vision series—a follow-up to WandaVision—stars Paul Bettany and is coming soon to Disney+.

Title: Marvel’s Vision Series Is Bringing Back James Spader as Ultron
Source: Gizmodo.com
Description: The as-yet-untitled Vision series—a follow-up to WandaVision—stars Paul Bettany and is coming to Disney+.

Title: The iPhone is getting a ‘glow’ up. What to expect from Apple’s Monday event
Source: CNN
Description: Apple excited fans with its vision for its “Apple Intelligence” AI artificial intelligence system“Apple Intelligence” artificial intelligence system earlier ...

Title: Crop loss is a huge problem for sustainability and food se

In [31]:
os.environ['NEWS_API_KEY'] = '6aaa6fe75c11492384c603cddab771ca'


In [55]:
import requests
import gradio as gr
import cohere
import pickle
import os

# Fetch the API keys from environment variables
cohere_api_key = os.getenv('COHERE_API_KEY')
news_api_key = os.getenv('NEWS_API_KEY')

# Initialize Cohere client
cohere_client = cohere.Client(cohere_api_key)

# Initialize a list to store the conversation history (up to 20 conversations)
conversation_history = []

# Function to fetch AI news using NewsAPI
def fetch_ai_news():
    url = "https://newsapi.org/v2/everything"
    parameters = {
        'q': 'artificial intelligence OR NLP OR computer vision',
        'sortBy': 'relevancy',
        'apiKey': news_api_key  # Securely use the API key from environment variables
    }

    response = requests.get(url, params=parameters)
    if response.status_code == 200:
        data = response.json()
        articles = data['articles']

        # Get the top 5 articles and format their titles
        news_summary = "\n".join([f"{i+1}. {article['title']} - {article['source']['name']}" for i, article in enumerate(articles[:5])])
        return f"Here are the top 5 AI news articles:\n{news_summary}"
    else:
        return "Sorry, I couldn't fetch the latest AI news at the moment."

# Function to generate a response using Cohere
def generate_response(conversation_history, user_input, cohere_client):
    # Limit the history to the last 20 exchanges (10 user-bot pairs)
    if len(conversation_history) > 40:  # Each user-bot pair counts as 2 entries
        conversation_history = conversation_history[-40:]

    # Build the conversation history prompt
    conversation_prompt = "\n".join(conversation_history)

    prompt = f"Previous conversation:\n{conversation_prompt}\n\nUser's question: {user_input}\n\nYour response:"

    response = cohere_client.generate(
        model='command-r-plus-04-2024',
        prompt=prompt,
        max_tokens=300,  # Increase the max_tokens to give a more complete response
        temperature=0.7
    )

    return response.generations[0].text.strip()

# Function to handle the chatbot conversation and include news
def chatbot(user_input):
    global conversation_history

    # Check if the user is asking for news
    if 'news' in user_input.lower() or 'trends' in user_input.lower():
        return fetch_ai_news()  # Fetch and return AI news

    # Retrieve the closest conversation based on the user's input
    closest_conversation = retrieve_closest_dialog(user_input, tokenizer, model, index, prompts, device)

    # Generate a response using Cohere
    bot_response = generate_response(conversation_history, user_input, cohere_client)

    # Append the current user input and bot response to the conversation history
    conversation_history.append(f"User: {user_input}")
    conversation_history.append(f"Bot: {bot_response}")

    return bot_response  # Return only the bot response (no need for a second output)

# Gradio Interface
def gradio_interface(user_input):
    return chatbot(user_input)

# Set up Gradio interface
interface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(lines=2, placeholder="Ask me anything..."),
    outputs="text",  # Single output for the bot's response
    title="AI Conversational Chatbot"
)

# Launch the Gradio app
interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d3675d4ed0f9d75236.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
